In [ ]:
# PLEASE READ:
# This notebook here is mostly compiling the results AFTER the benchmarks are
# done. If you use this notebook, please install LISA
#     https://github.com/ARM-software/lisa
# and then workload-processor
#     https://github.com/mrkajetanp/workload-processor.git
# first. After the installation, run this workload via:
# $ workload-processor run $WORKLOAD_PROCESSOR_INSTALL_DIR/agendas/agenda_pcmark.yaml baseline
# $ workload-processor run $WORKLOAD_PROCESSOR_INSTALL_DIR/agendas/agenda_pcmark.yaml test
# This will give you two directories of
#     pcmark_baseline_$RUNS_$DATE
# and
#     pcmark_test_$RUNS_$DATE
# To obtain power consumption results on the two benchmark runs, do
# $ workload-processor process pcmark_baseline_$RUNS_$DATE -m power
# and then
# $ workload-processor process pcmark_test_$RUNS_$DATE -m power
#
# Only after this should you start using this notebook. The reason is that
# these benchmark runs generate a huge amount of traces. Running the benchmarks
# inside this notebook can easily crash Jupyter completely.

In [ ]:
import os
import logging as log

from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, setup_notebook

setup_notebook()

In [16]:
pcmark = WorkloadNotebookAnalysis('/home/honxia02/spare_hdd/lisa', [
    'pcmark_mainroomuclamp_10_1110',
    'pcmark_sumroomdiv3.2_10_1110',
    'pcmark_sumroomdiv4_10_1110'
], label='PCMark Work 3')

pcmark.show()

,metric,value,unit,iteration,workload,id,wa_path,kernel,tag
0,PcmaDataManipulationV3Score,11264,,1,pcmark,wk1,pcmark_mainroomuclamp_10_1110,5.18.0-armpower-00029-gc35255a0e739,mainroomuclamp
1,PcmaVideoEditingV3Score,6788,,1,pcmark,wk1,pcmark_mainroomuclamp_10_1110,5.18.0-armpower-00029-gc35255a0e739,mainroomuclamp
2,PcmaWritingV3Score,13099,,1,pcmark,wk1,pcmark_mainroomuclamp_10_1110,5.18.0-armpower-00029-gc35255a0e739,mainroomuclamp
3,PcmaPhotoEditingV3Score,12895,,1,pcmark,wk1,pcmark_mainroomuclamp_10_1110,5.18.0-armpower-00029-gc35255a0e739,mainroomuclamp
4,PcmaWebV3Score,5951,,1,pcmark,wk1,pcmark_mainroomuclamp_10_1110,5.18.0-armpower-00029-gc35255a0e739,mainroomuclamp
...,...,...,...,...,...,...,...,...,...
175,PcmaWritingV3Score,12510,,10,pcmark,wk1,pcmark_sumroomdiv4_10_1110,5.18.0-armpower-00036-g7f9273f79788,sumroomdiv4
176,PcmaDataManipulationV3Score,11400,,10,pcmark,wk1,pcmark_sumroomdiv4_10_1110,5.18.0-armpower-00036-g7f9273f79788,sumroomdiv4
177,PcmaVideoEditingV3Score,6054,,10,pcmark,wk1,pcmark_sumroomdiv4_10_1110,5.18.0-armpower-00036-g7f9273f79788,sumroomdiv4
178,PcmaWorkv3Score,9319,,10,pcmark,wk1,pcmark_sumroomdiv4_10_1110,5.18.0-armpower-00036-g7f9273f79788,sumroomdiv4


benchmark_dirs: ['pcmark_mainroomuclamp_10_1110', 'pcmark_sumroomdiv3.2_10_1110', 'pcmark_sumroomdiv4_10_1110']
tags: ['mainroomuclamp', 'sumroomdiv3.2', 'sumroomdiv4']
kernels: ['5.18.0-armpower-00029-gc35255a0e739', '5.18.0-armpower-00036-g769ac0cc40ef', '5.18.0-armpower-00036-g7f9273f79788']


In [3]:
pcmark.plot.frequency_line()

In [ ]:
pcmark.plot.frequency_bar()

In [ ]:
pcmark.plot.results_line(['PcmaWebV3Score', 'PcmaDataManipulationV3Score', 'PcmaVideoEditingV3Score', 'PcmaPhotoEditingV3Score', 'PcmaWritingV3Score', 'PcmaWorkv3Score'])

In [6]:
pcmark.plot.power_meter_line()

In [17]:
pcmark.plot.power_meter_bar()

+------------+--------+----------------+-------+-----------+
|  channel   | metric |      tag       | value | perc_diff |
+------------+--------+----------------+-------+-----------+
|    CPU     | gmean  | mainroomuclamp | 230.9 |   0.0%    |
|  CPU-Big   | gmean  | mainroomuclamp | 89.1  |   0.0%    |
| CPU-Little | gmean  | mainroomuclamp | 124.2 |   0.0%    |
|  CPU-Mid   | gmean  | mainroomuclamp | 17.5  |   0.0%    |
|    GPU     | gmean  | mainroomuclamp | 40.2  |   0.0%    |
|   Total    | gmean  | mainroomuclamp | 271.1 |   0.0%    |
|    CPU     | gmean  | sumroomdiv3.2  | 217.4 |  -5.85%   |
|  CPU-Big   | gmean  | sumroomdiv3.2  | 72.7  |  -18.42%  |
| CPU-Little | gmean  | sumroomdiv3.2  | 124.4 |   0.16%   |
|  CPU-Mid   | gmean  | sumroomdiv3.2  | 20.3  |  15.73%   |
|    GPU     | gmean  | sumroomdiv3.2  | 40.0  |  -0.51%   |
|   Total    | gmean  | sumroomdiv3.2  | 257.4 |  -5.06%   |
|    CPU     | gmean  |  sumroomdiv4   | 204.8 |  -11.3%   |
|  CPU-Big   | gmean  | 

In [18]:
pcmark.plot.results_bar(['PcmaWebV3Score', 'PcmaDataManipulationV3Score', 'PcmaVideoEditingV3Score', 'PcmaPhotoEditingV3Score', 'PcmaWritingV3Score', 'PcmaWorkv3Score'])

+-----------------------------+----------------+--------+-----------+
|           metric            |      tag       | value  | perc_diff |
+-----------------------------+----------------+--------+-----------+
| PcmaDataManipulationV3Score | mainroomuclamp | 11.22k |   0.0%    |
| PcmaDataManipulationV3Score | sumroomdiv3.2  | 11.37k |   1.37%   |
| PcmaDataManipulationV3Score |  sumroomdiv4   | 11.29k |   0.69%   |
|   PcmaPhotoEditingV3Score   | mainroomuclamp | 13.65k |   0.0%    |
|   PcmaPhotoEditingV3Score   | sumroomdiv3.2  | 14.01k |   2.59%   |
|   PcmaPhotoEditingV3Score   |  sumroomdiv4   | 13.45k |  -1.48%   |
|   PcmaVideoEditingV3Score   | mainroomuclamp | 6531.1 |   0.0%    |
|   PcmaVideoEditingV3Score   | sumroomdiv3.2  | 6475.2 |  -0.86%   |
|   PcmaVideoEditingV3Score   |  sumroomdiv4   | 6583.2 |   0.8%    |
|       PcmaWebV3Score        | mainroomuclamp | 6078.1 |   0.0%    |
|       PcmaWebV3Score        | sumroomdiv3.2  | 6323.8 |   4.04%   |
|       PcmaWebV3Sco

In [ ]:
pcmark.plot.wakeup_latency_bar()

In [10]:
pcmark.plot.overutilized_line()

+--------------+----------+--------+------------+------------+
|    metric    |   tag    |  time  | total_time | percentage |
+--------------+----------+--------+------------+------------+
| overutilized | baseline | 61.27  |   466.19   |   13.14    |
| overutilized |  sumagg  | 106.43 |   465.03   |   22.89    |
+--------------+----------+--------+------------+------------+


In [12]:
pcmark.plot.wakeup_latency_cgroup_bar()

+-------------------+--------+----------+---------+-----------+
|      cgroup       | metric |   tag    |  value  | perc_diff |
+-------------------+--------+----------+---------+-----------+
|    background     | gmean  | baseline | 110.21μ |   0.0%    |
|    background     | gmean  |  sumagg  | 98.15μ  |  -10.95%  |
|    foreground     | gmean  | baseline | 155.26μ |   0.0%    |
|    foreground     | gmean  |  sumagg  | 119.4μ  |  -23.1%   |
| system-background | gmean  | baseline | 81.51μ  |   0.0%    |
| system-background | gmean  |  sumagg  | 76.29μ  |   -6.4%   |
+-------------------+--------+----------+---------+-----------+


In [15]:
pcmark.save_image_plots('/home/elitop01/uclamp_sum_agg/test_results/pcmark_sumagg_10_0910/plots_images')

In [15]:
main_avg_time = 464.6188862000001
sum_avg_time = 465.7076411
0.9415 * sum_avg_time / main_avg_time

0.9437062442332761